In [7]:
using Pkg; Pkg.activate(".");
using Convex
using SCS


  Activating new project at `~/Convex_Opt/6.S098-project`


7-element Vector{Float64}:
 0.37741520749211444
 2.883727608451974
 4.920941693455891
 3.3615395704430133
 1.0222990134394216
 3.9032055724634214
 0.3574933623363802

In [24]:
#CLASSES
class_numbers = Dict("6.100A"=>1, "6.100L"=>2, "6.042"=>3, "18.06"=>4, "18.C06"=>5, "6.3700"=>6, "6.3800"=>7,
"18.05"=>8, "6.009"=>9, "6.006"=>10, "6.3720"=>11, "6.3900"=>12, "6.3000"=>13, 
    "6.4110"=>14, "6.4400"=>15, "6.3100"=>16, "6.7201"=>17, "6.1220" => 18, "6.3950"=>19, "6.4590"=>20, "6.4120"=>21, "6.C35"=>22,
    "18.404"=>23, "6.3730"=>24, "6.5151"=>25, "6.5831" => 26, "6.8701"=>27, "6.4200"=>28, "6.4860"=>29, "6.8301"=>30, "6.8611"=>31, 
    "6.8371"=>32, "6.1800"=>33, "6.4210"=>34, "8.01"=>35, "8.02"=>36, "18.01"=>37, "18.02"=>38, "7.012" =>39, "3.091"=>40)


num_classes = 40
#BUILDING TAKEN
taken = zeros(40)
#GIRS SATISFIED
for i in 35:40
    taken[i] = 1
end


requirements = [([1, 2], 1), ([3], 1), ([4, 5], 1), ([6, 7, 8], 1), ([9], 1), ([10], 1), ([11, 12], 1), 
    ([13, 14, 15], 1), ([16, 14, 17], 1), ([18, 17, 15], 1), ([13, 14, 15], 1), ([19, 20, 21, 22], 1), 
    ([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22], 5), ([23, 24, 25, 26, 27, 32], 2),
    ([28, 29, 30, 31, 33, 34], 1), ([35, 36 ,37, 38, 39, 40], 6)]

remaining_reqs = Dict()

for (ind, req) in enumerate(requirements)
    remaining = req[2]
    for class in req[1]
        if taken[class] == 1
            remaining -= 1
            if remaining == 0
                break
            end
            
        end
        
    end
    remaining_reqs[ind] = remaining
end

remaining_reqs





Dict{Any, Any} with 16 entries:
  5  => 1
  16 => 0
  12 => 1
  8  => 1
  1  => 1
  6  => 1
  11 => 1
  9  => 1
  14 => 2
  3  => 1
  7  => 1
  4  => 1
  13 => 5
  15 => 1
  2  => 1
  10 => 1

In [27]:
c = 7*rand(num_classes)
time_vec = 7*ones(num_classes) + 7*rand(num_classes)
c

40-element Vector{Float64}:
 1.1391108491597546
 1.3927481839669749
 5.470904504057465
 5.488696411402272
 6.0349305282232795
 2.976904834376327
 2.64619846706188
 6.296384800653085
 5.821081979940469
 6.209625795839438
 6.534235424395592
 6.321692746061686
 2.614809345119018
 ⋮
 0.3720293803566581
 4.003959870502843
 3.3831403878557094
 0.20625928189455645
 0.22411918043819912
 2.473889451047344
 5.173419086011188
 3.2668225160548845
 6.776456731707294
 1.8166770823918532
 2.130709542446836
 6.996415633409068

In [68]:
x = Variable(num_classes)
objective = c'*x - 1*sum(x)
t = 40*6
constr = [
    x >= 0,
    x <= 1,
    time_vec'*x <= t
    
]
for (class, status) in enumerate(taken)
    
    if status == 1
        
        push!(constr, (x[class] == 0))    
    end   
end


for (ind, req) in enumerate(requirements)
    length = size(req[1])[1]
    
    req_array = Array{Convex.AbstractExpr}(undef, length)
    for i in 1:length[1]
        req_array[i] = x[req[1][i]]
        
    end
    push!(constr, (sum(req_array) >= remaining_reqs[ind]))
end
push!(constr, (x[4]+x[5] == 1))
push!(constr, (x[1] + x[2] == 1))
 

optimization = maximize(objective, constr)
solve!(optimization, SCS.Optimizer())
res = x.value
print(c'*res)

res





[107.5370107121235;;]------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 41, constraints m: 106
cones: 	  z: primal zero / dual free vars: 9
	  l: linear vars: 97
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 229, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 3.95e+02  7.86e+00  2.67e+02 -2.07e+02  1.00e-01  2.35e-04 
   225| 1.24e-03  2.37e-05  2.89e-04 -8.34e+01  2.14e-02  1.15e-03 
---------------

40×1 Matrix{Float64}:
 0.99999504821751
 4.985836971141371e-6
 1.000011229023546
 0.9999965328194862
 3.4971709804239247e-6
 1.000018344791814
 1.000022150411647
 1.0000192606579283
 1.0000093994487398
 1.0000093852302911
 1.0000165477553407
 1.000019832178153
 1.000024498062364
 ⋮
 2.530414063361388e-5
 1.0000231074297903
 1.0000163731353595
 1.6736307153481953e-5
 2.0796455136087078e-5
 1.000022267881409
 1.3875985148017292e-8
 1.6455190648880788e-8
 1.6241452884344416e-8
 1.6670908279444542e-8
 1.7014121597611673e-8
 1.716108881001057e-8

In [69]:
final_vector = round.(res, digits = 3)
print(final_vector)

[1.0; 0.0; 1.0; 1.0; 0.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 0.0; 1.0; 1.0; 1.0; 1.0; 0.1; 0.0; 1.0; 1.0; 1.0; 0.0; 0.0; 1.0; 1.0; 0.0; 1.0; 1.0; 0.0; 0.0; 1.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0;;]

In [70]:
#Reverse the dictionary so class index in key and class name is val
reverse_classes = Dict()
for (key, value) in class_numbers
    reverse_classes[value] = key
end
print(length(reverse_classes))
schedule = []
#GET LIST OF FINAL CLASSES
for i in 1:num_classes
    if final_vector[i] > .9999
        push!(schedule, reverse_classes[i])
    end
end

schedule
 

40

24-element Vector{Any}:
 "6.100A"
 "6.042"
 "18.06"
 "6.3700"
 "6.3800"
 "18.05"
 "6.009"
 "6.006"
 "6.3720"
 "6.3900"
 "6.3000"
 "6.4110"
 "6.3100"
 "6.7201"
 "6.1220"
 "6.3950"
 "6.C35"
 "18.404"
 "6.3730"
 "6.8701"
 "6.4200"
 "6.8301"
 "6.8611"
 "6.4210"

In [71]:
print(size(schedule))
time_vec'*final_vector

(24,)

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 239.98796827702574

In [72]:
c_sorted = []
for (ind, val) in enumerate(c)
    push!(c_sorted, (val, reverse_classes[ind]))
end
c_sorted
c_sorted = sort(c_sorted)

print("RATINGS ", c_sorted)


RATINGS Any[(0.01674107358514243, "6.4400"), (0.20625928189455645, "6.8371"), (0.22411918043819912, "6.1800"), (0.3720293803566581, "6.4860"), (1.1037562563817414, "6.5151"), (1.1391108491597546, "6.100A"), (1.2440241764393214, "6.5831"), (1.3927481839669749, "6.100L"), (1.8166770823918532, "18.02"), (2.130709542446836, "7.012"), (2.3487402422339105, "6.7201"), (2.473889451047344, "6.4210"), (2.614809345119018, "6.3000"), (2.64619846706188, "6.3800"), (2.7072060642300233, "6.4590"), (2.776022604987598, "6.4120"), (2.976904834376327, "6.3700"), (3.2668225160548845, "8.02"), (3.2994078409056566, "6.8701"), (3.3831403878557094, "6.8611"), (3.5814077129516733, "6.1220"), (3.770363367737921, "6.3950"), (4.003959870502843, "6.8301"), (4.434791516169647, "6.3100"), (4.541245269717419, "6.C35"), (4.802106023194701, "6.3730"), (5.173419086011188, "8.01"), (5.470904504057465, "6.042"), (5.488696411402272, "18.06"), (5.821081979940469, "6.009"), (5.884020141684222, "6.4110"), (6.0349305282232795,

In [73]:
t_sorted = []
for (ind, val) in enumerate(time_vec)
    push!(t_sorted, (val, reverse_classes[ind]))
end
t_sorted
t_sorted = sort(t_sorted)

print("TIME ", t_sorted)

TIME Any[(7.134249394233765, "6.5831"), (7.396059338516257, "6.4200"), (7.453847077624196, "6.4400"), (7.566455957170015, "6.100A"), (7.629599059188994, "6.4860"), (7.720090482611781, "6.3720"), (7.9027807886952015, "6.042"), (7.988289786870112, "6.8301"), (8.063216782709574, "6.1220"), (8.360355423630764, "6.5151"), (8.361240238143546, "6.7201"), (8.488635039417355, "18.06"), (8.541341885245481, "6.3900"), (8.724423998523866, "6.4210"), (9.286618824654433, "6.3000"), (9.395426143424102, "6.3800"), (9.79495084702973, "6.1800"), (9.826000265111922, "6.4110"), (9.925436963128544, "6.006"), (9.972179587490789, "6.009"), (10.046789284566916, "6.C35"), (10.340590565214246, "18.05"), (10.732325505245441, "3.091"), (11.311122268822817, "6.3700"), (11.433944186599682, "18.01"), (11.554575740671433, "7.012"), (11.55574881456855, "6.4590"), (11.794572657409711, "8.02"), (11.839478356831748, "18.02"), (12.028994228824299, "6.8611"), (12.571159178011918, "6.8371"), (12.575191723569695, "18.C06"), 